In [ ]:
# Isaac G Callison
# 6350-001
# Project 3
# Due 2/11/2020

In [ ]:
# imports
import random
from nltk.corpus import stopwords
from nltk.classify import NaiveBayesClassifier
from nltk import classify
from nltk.metrics import *
import collections
import heapq


In [ ]:
# pull in Sotu from file 
sotu_file = 'sotu.txt'
pres_by_party = 'presidentsbyparty.txt'

repub_list = [] ; dem_list = []

with open(pres_by_party) as prez:
    cl = prez.readline()
    cl = cl.strip()
    if (cl == 'Republican'):
        while cl != 'Democrat':
            cl = prez.readline()
            cl = cl.strip()
            if len(cl) > 1:
                cl = cl.lower()
                repub_list.append(cl)
    repub_list = repub_list[:-1]
    for line in prez:
        if len(line) > 1:
            line = line.strip()
            line = line.lower()
            dem_list.append(line)
# shift + tab shows documenation. 


In [ ]:
testFile = open("test.txt","w+")
stop_words = set(stopwords.words('english'))
with open(sotu_file) as sf:
    lines = sf.readlines()
    glob = [x.replace('\'', ',') for x in lines]
    glob = [x.lstrip('*') for x in glob]
    
    for line in glob:
        testFile.write(line)    
testFile.close()

In [ ]:
print(repub_list)

In [ ]:
r_file = open("rFile.txt", 'w+')
d_file = open("dFile.txt", 'w+')
f_file = open("f_file.txt", 'w+')
r_docs = []
d_docs = []
with open("test.txt") as tf:
    new_list = []
    lines = tf.readlines()
    lines = str(lines)
    lines = lines.split('@')
    #t = str(lines)
    #f_file.write(t)
    for r,item in enumerate(lines):
        if len(item) < 45:
            print("item: ",item)
        item = item.lower()
        if item in repub_list:
            #print(lines[r])
            r_file.write(lines[r])
            r_file.write('\n')
            r_file.write(lines[r+2])
            r_docs.append(lines[r+2])
            r_file.write('\n')
        elif item in dem_list:
            d_file.write(lines[r])
            d_file.write('\n')
            d_file.write(lines[r+2])
            d_docs.append(lines[r+2])
            d_file.write('\n')
        else:
            pass
            #f_file.write(lines[r])
# Clean Sotu, truncate to republican and dem and normalize data
r_file.close()
d_file.close()

In [ ]:
r_set = set()
r_dict = {}
with open("rFile.txt") as rf:
    r_lines = rf.readlines()
    for word in r_lines:
        word = word.split()
        for item in word:
            if item not in stop_words:
                item = item.lower()
                r_dict[item] = r_dict.get(item,0)+1 
                r_set.add(item)
    print(len(r_set))
    #print(r_dict)
        
r_freq = heapq.nlargest(100, r_dict, key=r_dict.get)
#print(r_freq)

In [ ]:
# Run classifier
def wordCount(word):
    return d_dict[word]

In [ ]:
d_set = set()
d_dict = {}
with open("dFile.txt") as df:
    d_lines = df.readlines()
    for word in d_lines:
        word = word.split()
        for item in word:
            if item not in stop_words:
                item = item.lower()
                d_set.add(item)
                d_dict[item] = d_dict.get(item,0)+1

d_freq = heapq.nlargest(100, d_dict, key=d_dict.get)
#print(d_freq)
#print()
#print(r_freq)

In [ ]:
repub_list = []
for k,v in r_dict.items():
    repub_list.append(k)

In [ ]:
dem_list = []
for k,v in d_dict.items():
    dem_list.append(k)
    
labeled_sotu = ([(word, 'democrat') for word in dem_list] + [(word, 'republican') for word in repub_list] )

In [ ]:
random.shuffle(labeled_sotu)

In [ ]:
# feature function - accepsta string(word) and returns a dictionary with key 'last_letter' and value the final character in the string
def sotu_features(word):
    bool = True
    if word in d_freq and word not in r_freq:
        w = "DEM"
    else:
        w="UNK"
    return {'freq': w }

In [ ]:
# construct a list consisting of the gender feature dictionaries: last letter of and the gender of a name
featuresets = [(sotu_features(n), party) for (n, party) in labeled_sotu]

print(len(featuresets), "data rows")

In [ ]:
train_len = int(len(featuresets) * .80)
test_len = (len(featuresets) - train_len)

In [ ]:
train_set, test_set = featuresets[train_len:], featuresets[:test_len]
#print(train_set)

In [ ]:
# using the training set to train the classifer

classifier = NaiveBayesClassifier.train(train_set)

In [ ]:
# run the entire test set, and print the classifier's accuracy

print(classify.accuracy(classifier, test_set))

In [ ]:
# show the features that provided the greatest distinction between male and female names
classifier.show_most_informative_features(10)